In [2]:
import pandas as pd
import requests

In [3]:
url="https://api.census.gov/data/2019/acs/acs1/subject?get=group(S2301)&for=school%20district%20(unified)&in=state:06"

In [4]:
response = requests.get(url)
response

<Response [200]>

In [8]:
response_json = response.json()

In [9]:
unifiedschool_df = pd.DataFrame(response_json,columns=response_json[0]).drop(0)
unifiedschool_df

,GEO_ID,NAME,S2301_C01_001E,S2301_C01_001EA,S2301_C01_001M,S2301_C01_001MA,S2301_C01_002E,S2301_C01_002EA,S2301_C01_002M,S2301_C01_002MA,...,S2301_C04_034E,S2301_C04_034EA,S2301_C04_034M,S2301_C04_034MA,S2301_C04_035E,S2301_C04_035EA,S2301_C04_035M,S2301_C04_035MA,state,school district (unified)
1,9700000US0632070,"Redlands Unified School District, California",103753,None,5033,None,6839,None,1649,None,...,5.7,None,4.6,None,3.4,None,2.2,None,06,32070
2,9700000US0632550,"West Contra Costa Unified School District, Cal...",205097,None,5334,None,11387,None,1736,None,...,1.8,None,1.0,None,3.2,None,1.6,None,06,32550
3,9700000US0633150,"Riverside Unified School District, California",215389,None,8763,None,21809,None,2711,None,...,3.5,None,2.3,None,1.4,None,1.0,None,06,33150
4,9700000US0633750,"Rowland Unified School District, California",87017,None,8025,None,5516,None,1515,None,...,2.1,None,1.8,None,6.0,None,3.1,None,06,33750
5,9700000US0633840,"Sacramento City Unified School District, Calif...",274468,None,9161,None,14299,None,2348,None,...,4.5,None,1.5,None,3.5,None,1.5,None,06,33840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,9700000US0627720,"Novato Unified School District, California",53039,None,2000,None,3541,None,925,None,...,6.0,None,4.9,None,1.3,None,1.5,None,06,27720
117,9700000US0628050,"Oakland Unified School District, California",353106,None,4191,None,15692,None,1922,None,...,4.5,None,1.7,None,2.9,None,0.9,None,06,28050
118,9700000US0628250,"Oceanside Unified School District, California",107834,None,5924,None,4459,None,1438,None,...,5.0,None,2.7,None,5.1,None,4.0,None,06,28250
119,9700000US0601332,"Twin Rivers Unified School District, California",138033,None,8194,None,9426,None,2080,None,...,5.4,None,2.6,None,5.9,None,3.7,None,06,01332


In [6]:
unifiedschool_df.loc[unifiedschool_df["NAME"]=="Hesperia Unified School District, California"]

,GEO_ID,NAME,S2301_C01_001E,S2301_C01_001EA,S2301_C01_001M,S2301_C01_001MA,S2301_C01_002E,S2301_C01_002EA,S2301_C01_002M,S2301_C01_002MA,...,S2301_C04_034E,S2301_C04_034EA,S2301_C04_034M,S2301_C04_034MA,S2301_C04_035E,S2301_C04_035EA,S2301_C04_035M,S2301_C04_035MA,state,school district (unified)
27,9700000US0600014,"Hesperia Unified School District, California",80008,None,5263,None,7595,None,1782,None,...,7.0,None,5.5,None,3.2,None,2.9,None,06,00014


## Rename columns

In [7]:
variableurl = "https://api.census.gov/data/2019/acs/acs1/subject/variables.json"

In [11]:
variables_json = requests.get(variableurl).json()

NameError: name 'variableurl' is not defined

In [9]:
variables_json["variables"]["S2301_C01_001E"]

{'label': 'Estimate!!Total!!Population 16 years and over',
 'concept': 'EMPLOYMENT STATUS',
 'predicateType': 'int',
 'group': 'S2301',
 'limit': 0,
 'attributes': 'S2301_C01_001EA,S2301_C01_001M,S2301_C01_001MA'}

In [10]:
new_labels = []
for col in unifiedschool_df.columns:
    label = variables_json["variables"].get(col)
    if label is not None:
        label = label['label']
    else:
        if col[-2:] == 'EA':
            label = variables_json["variables"].get(col[:-1])
            label = label['label'] + "||Annotation"
        elif col[-1] == 'M':
            label = variables_json["variables"].get(col[:-1]+'E')
            label = label['label'] + "||MarginOfError"
        elif col[-2:] == 'MA':
            label = variables_json["variables"].get(col[:-2]+'E')
            label = label['label'] + "||MarginOfErrorAnnotation"
    new_labels.append(label)

NameError: name 'variables_json' is not defined

In [11]:
assert len(new_labels) == len(unifiedschool_df.columns)

In [12]:
sum([1 for x in new_labels if x is None])

3

In [13]:
#new_labels[1] = 'NAME'
#new_labels[-2] = 'STATE'
#new_labels[-1] = 'SCHOOL DISTRICT (UNIFIED)'

In [14]:
unifiedschool_df.columns = new_labels

In [15]:
[col for col in unifiedschool_df.columns if "Annotation" not in col]

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
without_annotation_df = unifiedschool_df[[col for col in unifiedschool_df.columns if "Annotation" not in col]]
without_annotation_df

In [ ]:
without_annotation_df[[col for col in without_annotation_df.columns if 'Labor Force Participation Rate!!EDUCATIONAL ATTAINMENT' in col]]